In [ ]:
pip install python3-discogs-client
# or pip3 install python3-discogs-client

In [3]:
import discogs_client

In [8]:
# Replace the following user string and user_token first
# https://python3-discogs-client.readthedocs.io/en/latest/authentication.html
# ^ Check it and register an account on Discogs.com
d = discogs_client.Client('Your arbitrary user string here', user_token='your user token here')

In [9]:
class MyTrack:
    """
    One Instance represents one row in the final excel table.
    """
    def __init__(self, title, album):
        self.title = title
        self.track_artist = set()
        self.album = album
        self.labels = set()
        self.release_year = set()
        self.instruments = set()
        self.genre = set()
        self.format = set()
        self.countries_of_release = set()
        self.catalog_numbers = set()
        self.requires_manual_check = False

In [10]:
# 108475 is the artist id for Greg Phillinganes.
greg = d.artist(108475)
# All the releases/masters related to Greg Phillinganes.
greg_rel = greg.releases
# The release list can be paginated
greg_rel_page1 = greg_rel.page(0)
greg_rel_page5 = greg_rel.page(5)


In [11]:
# Final results will be stored in this dictionary
# Key: Each track's title
# Value: a MyTrack obj
track_dic = {}
# Name to be searched
artist_name = greg.name
# The artist instance
artist_obj = greg


def create_a_new_mytrack(cur_release, track_entry, is_release_main_artist, is_in_credits, main_release_name):
    """
    Create a new MyTrack instance based on the track_entry in the cur_release.
    :param cur_release: current version of the master
    :param track_entry: current track to create a new MyTrack instance
    :param is_release_main_artist: whether artist_obj is the main artist of cur_release
    :param is_in_credits: whether artist_name appears in the "Credits" part or not
    :param main_release_name: the title of the main_release of the master
    :return: title of the new track
    """
    if main_release_name == "":
        main_release_name = cur_release.title.strip()
    new_key = track_entry.title.strip() + " in " + main_release_name
    track_tmp = MyTrack(track_entry.title, main_release_name)
    for artist_entry in cur_release.artists:
        track_tmp.track_artist.add(artist_entry.name)
    for label_entry in cur_release.labels:
        track_tmp.labels.add(label_entry.name)
        track_tmp.catalog_numbers.add(label_entry.data['catno'])
    track_tmp.release_year.add(cur_release.year)
    if is_release_main_artist:
        track_tmp.instruments.add("Main")
    if is_in_credits:
        for extra_artist_entry in cur_release.data['extraartists']:
            if extra_artist_entry['name'] == artist_name:
                track_tmp.instruments.add(extra_artist_entry['role'])
                if extra_artist_entry['tracks'] != "":
                    track_tmp.requires_manual_check = True
                break
    track_tmp.genre.update(cur_release.genres)
    for format_entry in cur_release.formats:
        track_tmp.format.add(format_entry['name'])
    track_tmp.countries_of_release.add(cur_release.country)
    track_tmp.duration = track_entry.duration
    if artist_obj in track_entry.credits:
        new_role = ""
        for ex_artist_entry in track_entry.data['extraartists']:
            if ex_artist_entry['name'] == artist_name:
                new_role = ex_artist_entry['role']
        track_tmp.instruments.add(new_role)
    track_dic[new_key] = track_tmp
    return track_entry.title


def update_a_mytrack(cur_release, track_entry, main_release_name):
    """
    Update an existing MyTrack instance in the track_dic, based on the track_entry in the cur_release.
    :param cur_release: current version of the master
    :param track_entry: current track to create a new MyTrack instance
    :param main_release_name: the title of the main_release of the master
    :return: the title of the updated MyTrack instance
    """
    if main_release_name != "":
        new_key = track_entry.title.strip() + " in " + main_release_name
    else:
        new_key = track_entry.title.strip() + " in " + cur_release.title.strip()
    track_cur = track_dic.get(new_key)

    for label_entry in cur_release.labels:
        track_cur.labels.add(label_entry.name)
        track_cur.catalog_numbers.add(label_entry.data['catno'])
    track_cur.countries_of_release.add(cur_release.country)
    track_cur.release_year.add(cur_release.year)
    for format_entry in cur_release.formats:
        track_cur.format.add(format_entry['name'])
    return track_entry.title


def check_single_track_credit(cur_release, track_entry, artist_t, is_release_main_artist, is_in_credits, main_release_name):
    """
    If the artist_obj is not the main artist of the release, then check artist_obj's contribution in a single track.
    :param cur_release: current version of the master
    :param track_entry: current track to create a new MyTrack instance
    :param artist_t: artist instance to be found
    :param is_release_main_artist: whether artist_obj is the main artist of cur_release
    :param is_in_credits: whether artist_name appears in the "Credits" part or not
    :param main_release_name: the title of the main_release of the master
    :return: title of the current track, whether updated
    """
    new_key = track_entry.title.strip() + " in " + main_release_name
    if artist_t in track_entry.credits:
        if new_key in track_dic.keys():
            cur_track_title = update_a_mytrack(cur_release, track_entry, main_release_name)
        else:
            cur_track_title = create_a_new_mytrack(cur_release, track_entry, is_release_main_artist, is_in_credits, main_release_name)
        new_role = ""
        for ex_artist_entry in track_entry.data['extraartists']:
            if ex_artist_entry['name'] == artist_name:
                new_role = ex_artist_entry['role']
        track_dic[new_key].instruments.add(new_role)
        return cur_track_title, True
    return "", False


def get_info_by_single_track(cur_release, is_release_main_artist, is_in_credits, track_title_list, main_release_name):
    """
    If the artist_obj is not the main artist of the release, then check artist_obj's contribution track by track.
    :param track_title_list: list of created MyTrack instances' titles
    :param cur_release: current version of the master
    :param is_release_main_artist: whether artist_obj is the main artist of cur_release
    :param is_in_credits: whether artist_name appears in the "Credits" part or not
    :param main_release_name: the title of the main_release of the master
    :return: null
    """
    for track_e in cur_release.tracklist:
        cur_track_title, is_updated = check_single_track_credit(cur_release, track_e, artist_obj, is_release_main_artist, is_in_credits, main_release_name)
        if is_updated:
            track_title_list.append(cur_track_title)


def get_info_from_release(cur_release, main_release_name):
    """
    Complete data retrieving in one version/release.
    :param cur_release: current version of the master
    :param main_release_name: the title of the main_release of the master
    :return: list of created MyTrack instances' titles, whether current artist is the main artist || current artist appear in "Credits"
    """
    is_release_main_artist = False
    is_in_credits = False
    track_title_list = []
    if greg in cur_release.artists:
        is_release_main_artist = True
    if greg in cur_release.credits:
        is_in_credits = True
    if is_release_main_artist or is_in_credits:
        for track_entry in cur_release.tracklist:
            new_key = track_entry.title.strip() + " in " + main_release_name
            if new_key in track_dic.keys():
                update_a_mytrack(main_release_tmp, track_entry, main_release_name)
            else:
                track_title_list.append(create_a_new_mytrack(main_release_tmp, track_entry, is_release_main_artist, is_in_credits, main_release_name))

        return track_title_list, True
    else:
        get_info_by_single_track(cur_release, is_release_main_artist, is_in_credits, track_title_list, main_release_name)
        return track_title_list, False

def update_from_other_release_version(cur_release, track_title_list, update_all, main_release_name):
    """
    Update MyTrack in versions which are not main_release/version. 
    :param cur_release: current version of the master
    :param track_title_list: list of created MyTrack instances' titles
    :param update_all: whether current artist is the main artist || current artist appear in "Credits"
    :param main_release_name: the title of the main_release of the master
    :return: null
    """
    is_release_main_artist = False
    is_in_credits = False
    if greg in cur_release.artists:
        is_release_main_artist = True
    if greg in cur_release.credits:
        is_in_credits = True
    for track_entry in cur_release.tracklist:
        if track_entry.title in track_title_list:
            # print(track_title_list)
            update_a_mytrack(cur_release, track_entry, main_release_name)
        elif update_all:
            track_title_list.append(create_a_new_mytrack(cur_release, track_entry, is_release_main_artist, is_in_credits, main_release_name))
        check_single_track_credit(cur_release, track_entry, artist_obj, is_release_main_artist, is_in_credits, main_release_name)

# Main program starts here
for cur_rel in greg_rel_page1:
    # Each release's type is other master or release
    if cur_rel.data["type"] == "master":
        main_release_tmp = cur_rel.main_release
        # We first check the main_release of the master
        track_names, should_update_all = get_info_from_release(main_release_tmp, main_release_tmp.title.strip())
        # Then we iterate through other versions, mainly to collect new Countries of Release, Catalog #, Year,
        for version_entry in cur_rel.versions:
            update_from_other_release_version(version_entry, track_names, should_update_all, main_release_tmp.title.strip())
    else:
        track_names = get_info_from_release(cur_rel, cur_rel.title.strip())
    print(track_names)


['Girl Talk', 'Maxxed Out ']
['Baby, I Do Love You', 'Do It All For Love']
["Takin' It Up All Night", "Takin' It Up All NIght", "I Don't Want To Be The One "]
['Girl Talk', 'Baby I Do Love You', 'Takin It Up All Night', 'Forever Now', 'Big Man', 'I Dont Want To Be The One', 'Maxxed Out', 'Do It All For Love', 'The Call', 'Baby, I Do Love You', "Takin' It Up All Night", "I Don't Want To Be The One"]
['Behind The Mask', "Won't Be Long Now", "Playin' With Fire", 'I Have Dreamed', 'Come As You Are', 'Lazy Nina', 'Signals', 'Countdown To Love', 'Shake It', 'Contdown To Love', 'Bonus Tracks', 'Only You', 'Behind The Mask (12" Version)', 'Playin\' With Fire (12" Version)', 'Behind The Mask (Instrumental Version)', "Playin' With Fire (Instrumental Version)", 'Behind The Mask (Single Version)', 'Behind The Mask (12" Ver.)', 'Playin\' With Fire (12" Ver.)', 'Behind The Mask (Instrumental Ver.)', "Playin' With Fire (Instrumental Ver.)", 'Behind The Mask (Single Ver.)']
(['Lazy Nina', 'Only You'],

['A Song For You', 'When Love Comes To Town', 'I Just Called To Say I Love You']
(['Silent Night', 'Song Of Mary (Holy Is The Lord)', 'O, Holy Night', 'Let It Snow', 'I Hear Angels Calling', 'Hark! The Herald Angels Sing', 'Do You Hear What I Hear', 'Away In A Manger', 'O Come, O Come, Emmanuel', 'Song Of Mary (Holy Is The Lord) Instrumental'], True)
['You And I', 'Chuva', 'Ocean Drive', 'Baker Street', 'Angelina', 'Memphis', "That's When You Save Me", 'Ladyland', "Shout About Ya'", 'Nostalgia']
['Introduction By Claude Nobs', 'The Good, The Bad And The Ugly', 'Let The Good Times Roll', "Makin' Whoopee", "Moody's Mood", 'The Television Song', 'Honeysuckle Rose', 'Shiny Stockings', "I'm Gonna Move To The Outskirts Of Town / In The Heat Of The Night", 'If I Ever Lose This Heaven', 'Midnight Sun', 'One Mint Julep', "Goin' To Chicago Blues", 'My Ship / Summertime', 'Smoke Gets In Your Eyes', 'Almost Like Being In Love', "Miss Celie's Blues (Sister)", "What's Going On", "Eyes Of Love (Carol

['At Last My Search Is Over', 'If I Could Bring Back Yesterday', 'Midnight Flight To Your Love', 'Since I Found Love', 'Happiness', 'Uncle James', 'Tomorrow Child']
['Sexasonic', 'Mr. Sin', 'Paradise', 'Good To Me', 'When I Get Back Home', 'Leaving You Is Killing Me', 'To Make You Stay', 'Leave Your Spirit Behind', 'Bye, Bye, Baby', 'Ghetto Penthouse', 'Bonus Tracks', 'Leaving You Is Killing Me (Single Version)', 'Sexasonic (Single Version)']
['Erika', 'Let Love Flow For Peace', 'So Long', 'I Heard The Voice Of Music Say', 'Mood I "L.O.V.E. Love"', 'Fantasy', "Don't Overlook The Feelings", 'Mood II "Love At First Sight"', 'Rhythm Of Our Souls', 'Mood III "Oliver"', 'Mood 1 "L.O.V.E. Love"', 'Don’t Overlook The Feelings', 'Mood 2 "Love At First Sight"', 'Mood 3 "Oliver"', 'Erika (Extended Mix)']
['One To One', "I Don't Know", 'Rest Yourself', 'I Too Am Wanting', 'Tiki Tiki Donga', "Don't Cry", 'Harmour Love', 'One To One (Reprise)', 'Uno A Uno', 'No Se', 'Des-Cansa', 'Yo Tambien Deseo',

In [12]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')


In [13]:
def convert_to_df(track_dic):
    """
    Convert data into a dataframe. 
    :param track_dic: final results dictionary
    :return: df: dataframe
    """
    all_tracks = []

    for i in track_dic:
        new_intance = {}
        for j in track_dic[i].__dict__:
            
            if type(track_dic[i].__dict__[j]) is str or type(track_dic[i].__dict__[j]) is bool:
                new_intance[j] = track_dic[i].__dict__[j]
            # convert set to string, split by ','
            else:
                new_intance[j] = ', '.join(str(v) for v in track_dic[i].__dict__[j])
        all_tracks.append(new_intance)
    df = pd.DataFrame(all_tracks)
    # replace empty cells to NAN
    df = df.replace('', np.NaN)
    return df

In [14]:
df = convert_to_df(track_dic)

In [15]:
df

,title,track_artist,album,labels,release_year,instruments,genre,format,countries_of_release,catalog_numbers,requires_manual_check,duration
0,Girl Talk,Greg Phillinganes,Girl Talk,Planet (15),1981,"Main, Producer","Funk / Soul, Electronic",Vinyl,US,"P-47938, P-49738",False,3:43
1,Maxxed Out,Greg Phillinganes,Girl Talk,Planet (15),1981,Main,Rock,Vinyl,US,P-49738,False,5:02
2,"Baby, I Do Love You",Greg Phillinganes,"Baby, I Do Love You",Planet (15),1981,"Main, Producer","Funk / Soul, Electronic",Vinyl,US,"P-47928, AS-11493, P-47928-A",False,4:15
3,Do It All For Love,Greg Phillinganes,"Baby, I Do Love You",Planet (15),1981,"Main, Producer","Funk / Soul, Electronic",Vinyl,US,P-47928-A,False,4:26
4,Takin' It Up All Night,Greg Phillinganes,Takin' It Up All Night,Planet (15),1981,"Main, Producer","Rock, Funk / Soul",Vinyl,US,"AS-11514, P-47935",False,4:50
...,...,...,...,...,...,...,...,...,...,...,...,...
687,Let Him In / Walk Softly / Dancin' To Keep Fro...,Maxi Anderson,Maxi,Blue Note,1977,Keyboards,Funk / Soul,8-Track Cartridge,US,BN-LA738-H,False,10:12
688,Dancin' To Keep From Cryin' (cont.) / This One...,Maxi Anderson,Maxi,Blue Note,1977,Keyboards,Funk / Soul,8-Track Cartridge,US,BN-LA738-H,False,10:11
689,"The Perfect Day / Delta Road / Glory, Glory",Maxi Anderson,Maxi,Blue Note,1977,Keyboards,Funk / Soul,8-Track Cartridge,US,BN-LA738-H,False,10:13
690,"Glory, Glory (cont.) / By Your Side / Music On...",Maxi Anderson,Maxi,Blue Note,1977,Keyboards,Funk / Soul,8-Track Cartridge,US,BN-LA738-H,False,10:11


In [19]:
# convert dataframe to csv file
import os  
os.makedirs('output', exist_ok=True)
df.to_csv('output/sample_output_Greg_Phillinganes.csv') 

In [20]:
# convert dataframe to xlsx file
# pip install openpyxl
df.to_excel("output/sample_output_Greg_Phillinganes.xlsx")